In [ ]:
import os

import simuran
import numpy as np

In [ ]:
# Establish paths
data_start_path = r"D:\SubRet_recordings_imaging\CSubRet1\CSubRet1_recording\CSR1_small sq"
data_out_path = r"E:\Dev\nscience"

# Resources
1. Check out https://github.com/seankmartin/lfp_atn for code, notes about LFP papers, and notes on the paper.
2. Check out https://github.com/seankmartin/SIMURAN for further code.

Last updated on 16/04/2021

# Working with SIMURAN
The basic object in SIMURAN is a Recording. A Recording should hold everything associated with a single Ephys recording session, all the signals (e.g. LFP), spatial information, spike times, stimulation info, etc.

However, it is rare that one only wants to analyse a single recording. To better facilate this, SIMURAN has a RecordingContainer - think of it like a list of Recording instances, but with some extra functionality.

Below are examples of working with this

In [ ]:
# Working with data via a CSV index
loader_kwargs = {"system": "Axona", "pos_extension": ".txt"}
df = simuran.index_ephys_files(
    data_start_path,
    loader_name="neurochat",
    out_loc=os.path.join(data_out_path, "index.csv"),
    post_process_fn=None,
    overwrite=False,
    post_process_kwargs=None,
    loader_kwargs=loader_kwargs)

def recording_from_df_line(line):
    params = {
        "loader": "neurochat",
        "loader_kwargs": loader_kwargs,
    }
    filename = os.path.join(line.filename, line.folder)
    recording = simuran.Recording(params=params, base_file=filename, load=True)

    return recording

# for example, load the first file in the list
recording = recording_from_df_line(df.iloc[0])
print(recording)

# or load all files from a subset of the list
recordings = simuran.RecordingContainer()
for row in df.head().itertuples(index=False):
    recordings.append(recording_from_df_line(row))
print(recordings)

In [ ]:
# Working with data via param files - see simuran/params for example
def recording_info():
    def setup_signals():
        """Set up the signals (such as eeg or lfp)."""
        # The total number of signals in the recording
        num_signals = 32

        # What brain region each signal was recorded from
        regions = ["RSC"] * 2 + ["SUB"] * 30

        # If the wires were bundled, or any other kind of grouping existed
        # If no grouping, groups = [i for i in range(num_signals)]
        groups = [0, 0, 1, 1]
        for i in range(2, 9):
            groups.append(i)
            groups.append(i)
            groups.append(i)
            groups.append(i)

        # The sampling rate in Hz of each signal
        sampling_rate = [250] * num_signals
        channel_type = ["eeg"] * num_signals

        # This just passes the information on
        output_dict = {
            "num_signals": num_signals,
            "region": regions,
            "group": groups,
            "sampling_rate": sampling_rate,
            "channel_type": channel_type,
        }

        return output_dict

    def setup_units():
        """Set up the single unit data."""
        # The number of tetrodes, probes, etc - any kind of grouping
        num_groups = 8

        # The region that each group belongs to
        regions = ["SUB"] * num_groups

        # A group number for each group, for example the tetrode number
        groups = [1, 2, 3, 4, 9, 10, 11, 12]

        output_dict = {
            "num_groups": num_groups,
            "region": regions,
            "group": groups,
        }

        return output_dict

    def setup_spatial():
        """Set up the spatial data."""
        arena_size = "default"

        output_dict = {
            "arena_size": arena_size,
        }
        return output_dict

    def setup_loader():
        """
        Set up the loader and keyword arguments for the loader.

        See also
        --------
        simuran.loaders.loader_list.py

        """
        # The type of loader to use, see simuran.loaders.loader_list.py for options
        # For now nc_loader is the most common option
        # loader = "params_only"
        loader = "nc_loader"

        # Keyword arguments to pass to the loader.
        loader_kwargs = {
            "system": "Axona",
        }

        output_dict = {
            "loader": loader,
            "loader_kwargs": loader_kwargs,
        }

        return output_dict

    load_params = setup_loader()

    mapping = {
        "signals": setup_signals(),
        "units": setup_units(),
        "spatial": setup_spatial(),
        "loader": load_params["loader"],
        "loader_kwargs": load_params["loader_kwargs"],
    }
    return mapping
recording = simuran.Recording(params=recording_info(), base_file=set_file_location)
print(recording)

# Working with multiple recordings by parameter files
This does not lend itself to working in notebook format very well. All parameter file examples are in simuran/params
The idea is that you:
1. Establish a mapping file - this looks like above, and just outlines what should be present in the recording and some extra metadata. See simuran_base_params.py.
2. Establish a set of files to load using this mapping file. This can be achieved by manually listing them, or by regular expression. See simuran_batch_params.py
3. Establish a set of functions that will be run on the files, and what will be saved from these. See simuran_fn_params.py.

At this point, you can then define a batch of (1 - mapping files, 2 - recordings to load, 3 - functions to run). This file would describe a list of (1, 2, 3) groups, and optionally have functions to run at the end. For example, one could run analysis on all rats in a t-maze, and then perform a summary plot. See simuran_multi_params.py.

# Results so far
See simuran/examples/single_axona_recording.py for some idea on this.

## Plotting the signals and average
![signals](images/lfp-signals-crop.PNG)

## There appears to be clusters of signals (why?)
![difference](images/lfp-difference.PNG)

## Evaluating the effect of filtering
![filtering](images/filt_comp-Lsubret5-01122017.PNG)